In [91]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import gym

import torch
import json
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
import cv2
import os

import random                 # Handling random number generation
import time                   # Handling time calculation
from skimage import transform # Help us to preprocess the frames

from collections import deque # Ordered collection with ends
import matplotlib.pyplot as plt  # Display graphs

import warnings                  # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore')


In [92]:
### MODEL HYPERPARAMETERS
learning_rate =  1e-5    # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 100000         # Total episodes for training

batch_size = 128

# FIXED Q TARGETS HYPERPARAMETERS
max_tau = 1000 #Tau is the C step where we update our target network
update_steps = 4

# EXPLORATION HYPERPARAMETERS for epsilon greedy strategy
explore_start = 0.1           # exploration probability at start
explore_stop = 0.0001            # minimum exploration probability        # exponential decay rate for exploration prob
explore = 20000

clip_norm = 0.0001

model_folder = f"./models/prioritised_replay/explore/EX{explore_stop}_B{batch_size}_CN{clip_norm}_LR{learning_rate}"
os.makedirs(model_folder)
log_path = f"./logs/prioritised_replay/explore/EX{explore_stop}_B{batch_size}_CN{clip_norm}_LR{learning_rate}.txt"

# Q LEARNING hyperparameters
gamma = 0.99               # Discounting rate

### MEMORY HYPERPARAMETERS
## If you have GPU change to 1million
pretrain_length = 1000             # Number of experiences stored in the Memory when initialized for the first time
memory_size = 50000 ##100000                 # Number of experiences the Memory can keep

env = gym.make('CartPole-v1')



In [93]:
class DDDQNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(4, 64)
        self.relu = nn.ReLU()

        self.fc_value = nn.Linear(64, 256)
        self.value = nn.Linear(256, 1)

        self.fc_adv = nn.Linear(64, 256)
        self.adv = nn.Linear(256, 2)

    def forward(self, x):
        y = self.relu(self.fc1(x))
        value = self.relu(self.fc_value(y))
        adv = self.relu(self.fc_adv(y))

        value = self.value(value)
        adv = self.adv(adv)

        output = value + adv - torch.mean(adv, dim=1, keepdim=True)

        return output

    def select_action(self, x):
        with torch.no_grad():
            Q = self.forward(x)
            action_index = torch.argmax(Q, dim=1)
        return action_index.item()
    
class SumTree(object):
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
    
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
        
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update (tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
    
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node
    
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        """
        Remember that our tree is composed of a sum tree that contains the priority scores at his leaf
        And also a data array
        We don't use deque because it means that at each timestep our experiences change index by one.
        We prefer to use a simple array and to overwrite when the memory is full.
        """
        self.tree = SumTree(capacity)
        
    """
    Store a new experience in our tree
    Each new experience have a score of max_prority (it will be then improved when we use this exp to train our DDQN)
    """
    def store(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        
        # If the max priority = 0 we can't put priority = 0 since this exp will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)   # set the max p for new p

        
    """
    - First, to sample a minibatch of k size, the range [0, priority_total] is / into k ranges.
    - Then a value is uniformly sampled from each range
    - We search in the sumtree, the experience where priority score correspond to sample values are retrieved from.
    - Then, we calculate IS weights for each minibatch element
    """
    def sample(self, n):
        # Create a sample array that will contains the minibatch
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment
    
        # Here we increasing the PER_b each time we sample a new minibatch
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])  # max = 1
        
        # Calculating the max_weight
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            """
            A value is uniformly sample from each range
            """
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            """
            Experience that correspond to each value is retrieved
            """
            index, priority, data = self.tree.get_leaf(value)
            
            #P(j)
            sampling_probabilities = priority / self.tree.total_priority
            
            #  IS = (1/N * 1/P(i))**b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PER_b)/ max_weight
                                   
            b_idx[i]= index
            
            experience = [data]
            
            memory_b.append(experience)
        
        return b_idx, memory_b, b_ISWeights
    
    """
    Update the priorities on the tree
    """
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)


def update_target_params(PolicyNetwork, TargetNetwork):
    """
    Copy parameters of the Policy network to Target network.
    """
    TargetNetwork_state_dict = TargetNetwork.state_dict()
    PolicyNetwork_state_dict = PolicyNetwork.state_dict()
    for key in PolicyNetwork_state_dict:
        TargetNetwork_state_dict[key] = PolicyNetwork_state_dict[key]
        TargetNetwork.load_state_dict(TargetNetwork_state_dict)

    return PolicyNetwork, TargetNetwork   

In [94]:
import random
# Instantiate memory
memory = Memory(memory_size)
import numpy as np 
from skimage import transform

env = gym.make('CartPole-v1')
n_state = env.observation_space.shape[0]  # 4
n_action = env.action_space.n

state, _ = env.reset()


for i in range(pretrain_length):
    
    action = random.randint(0,1)
    next_state, reward, done, _, _ = env.step(action)

    if done:
        next_state = np.zeros(n_state)
        experience = state, action, reward, next_state, done
        #print("shape of a stack frame of dead: ", state.shape)
        memory.store(experience)
        
        # Start a new episode
        state, _ = env.reset()
    else:
        experience = state, action, reward, next_state, done
        memory.store(experience)

        state = next_state

env.close()


In [95]:
# Instantiate the DQNetwork
PolicyNetwork = DDDQNet()

# Instantiate the target network
TargetNetwork = DDDQNet()

optimizer = optim.RMSprop(PolicyNetwork.parameters(), lr=learning_rate)

def select_action(epsilon, state):
    """
    This function will do the part
    With ϵ select a random action atat, otherwise select at=argmaxaQ(st,a)

    Input state is 4 stacked states. 
    """

    num = random.random()

    if (num < epsilon):
        action = random.randint(0, 1)
        return action
    # Greedy action
    else:
        
        tensor_state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        action = PolicyNetwork.select_action(tensor_state)
        return action

epsilon = explore_start
# Set Target network params
PolicyNetwork, TargetNetwork = update_target_params(PolicyNetwork, TargetNetwork)

out_f = open(log_path, 'w')

env = gym.make('CartPole-v1')

total_steps = 0

for episode in range(total_episodes):

    episode_step = 0
    episode_reward = 0
    state, _ = env.reset()
    # print("state: ", state)
    

    for time_steps in range(10000):

        episode_step += 1
        total_steps += 1

        action = select_action(epsilon, state)
        next_state, reward, done, _, _ = env.step(action)

        episode_reward += reward

        if done: 
            next_state = np.zeros(n_state)
            experience = state, action, reward, next_state, done
            memory.store(experience)

        else: 
            experience = state, action, reward, next_state, done
            # print("state: ", state)
            memory.store(experience)
            state = next_state


        tree_idx, batch, ISWeights_mb = memory.sample(batch_size)
        
        batch_states = torch.FloatTensor([each[0][0] for each in batch] )

        batch_actions = torch.FloatTensor([each[0][1] for each in batch]).unsqueeze(1)
        batch_rewards = torch.FloatTensor([each[0][2] for each in batch]).unsqueeze(1)
        batch_next_states = torch.FloatTensor([each[0][3] for each in batch] ) # stacked frames of np arrays 
        batch_dones = torch.FloatTensor([each[0][4] for each in batch]).unsqueeze(1)

        actions_index = batch_actions.detach().numpy().flatten()

        with torch.no_grad():
            policy_q_next = PolicyNetwork(batch_next_states)
            target_q_next = TargetNetwork(batch_next_states)
            online_max_action = torch.argmax(policy_q_next, dim=1, keepdim=True)
            y = batch_rewards + (1 - batch_dones) * gamma * target_q_next.gather(1, online_max_action.long())

        loss = F.mse_loss(PolicyNetwork(batch_states).gather(1, batch_actions.long()), y)
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_value_(PolicyNetwork.parameters(), clip)
        torch.nn.utils.clip_grad_norm_(PolicyNetwork.parameters(), clip_norm)
        optimizer.step()

        terminal_np = batch_dones.detach().numpy().flatten()
        policy_q_next_np = policy_q_next.detach().numpy()
        target_q_next_np = target_q_next.detach().numpy()
        rewards_np = batch_rewards.detach().numpy().flatten()

        target_qs_batch = []

        for i in range(0, len(batch)):

            terminal = terminal_np[i]
            action = np.argmax(policy_q_next_np[i])

            if terminal:
                target_qs_batch.append(torch.tensor(rewards_np[i], dtype=torch.float32)) # rewards_mb[i] is a TENSOR

            else: 
                target = rewards_np[i] + gamma * target_q_next_np[i][action]

                target_qs_batch.append(torch.tensor(target, dtype=torch.float32))

        ACTIONS = [torch.tensor(np.array([0,1]), dtype=torch.float32), torch.tensor(np.array([1,0]), dtype=torch.float32)]
        predicted_qs = [torch.sum(torch.mul(policy_q_next[i], ACTIONS[int(actions_index[i])])) for i in range(batch_size)]

        absolute_errors = [torch.abs(torch.subtract(predicted_qs[i], target_qs_batch[i])) for i in range(batch_size)]
        
        absolute_errors_np = np.array([error.item() for error in absolute_errors])
        ISWeights_list = ISWeights_mb.tolist()
        memory.batch_update(tree_idx, absolute_errors_np)


        if epsilon > explore_stop:
            epsilon -= (explore_start - explore_stop) / explore

        # print("POLICY PARAMS: " , PolicyNetwork.state_dict())
        if total_steps % update_steps == 0:
            PolicyNetwork, TargetNetwork = update_target_params(PolicyNetwork, TargetNetwork)
            # print("Model updated")

        if done: 
            # Write batch stats to log files 
            loss_at_end_of_episode = torch.mean(torch.stack(absolute_errors), dim=0).item()

            out_f.write(json.dumps({
                'episode': episode,
                'reward': episode_reward,
                'total_steps': total_steps,
                'length': episode_step,
                'loss': loss_at_end_of_episode,
                'epsilon': epsilon,
            }) + '\n')

            out_f.flush()
            

            print(f"Episode {episode}     |      Reward: {episode_reward}     |     length: {episode_step}      |      Loss: {loss_at_end_of_episode}    |     Total timesteps: {total_steps}")
        
            if episode % 100 == 0:
                torch.save(PolicyNetwork, f"{model_folder}/E_{episode}.pt")
                print(f"======Model saved.======")
            
            break

    if total_steps > 100000:
        break


env.close()






Episode 0     |      Reward: 9.0     |     length: 9      |      Loss: 1.039174199104309    |     Total timesteps: 9
======Model saved.======
Episode 1     |      Reward: 10.0     |     length: 10      |      Loss: 1.0439499616622925    |     Total timesteps: 19
Episode 2     |      Reward: 9.0     |     length: 9      |      Loss: 1.0415340662002563    |     Total timesteps: 28
Episode 3     |      Reward: 9.0     |     length: 9      |      Loss: 1.0489113330841064    |     Total timesteps: 37
Episode 4     |      Reward: 9.0     |     length: 9      |      Loss: 1.0449120998382568    |     Total timesteps: 46
Episode 5     |      Reward: 13.0     |     length: 13      |      Loss: 1.0551142692565918    |     Total timesteps: 59
Episode 6     |      Reward: 9.0     |     length: 9      |      Loss: 1.06353759765625    |     Total timesteps: 68
Episode 7     |      Reward: 9.0     |     length: 9      |      Loss: 1.067368507385254    |     Total timesteps: 77
Episode 8     |      Rew